### Parallelism of static circuits
This tutorial is an overview of how to use Bloqade's parallelism tools to optimize your circuit for parallel execution, as well as give you an intuition for what makes a ``good" program via analysis with our noise models.

In [21]:
import bloqade.cirq_utils as utils
import cirq

Lets kick things off with a simple example: a linear and log depth GHZ state preparation circuit.

In [ ]:
def build_linear_ghz(n_qubits:int)->cirq.Circuit:
    qubits = cirq.LineQubit.range(n_qubits)
    circuit = cirq.Circuit()
    circuit.append(cirq.H(qubits[0]))
    for i in range(n_qubits-1):
        circuit.append(cirq.CNOT(qubits[i], qubits[i+1]))
    return circuit

def build_log_ghz(n_qubits:int)->cirq.Circuit:
    qubits = cirq.LineQubit.range(n_qubits)
    circuit = cirq.Circuit()
    circuit.append(cirq.H(qubits[0]))
    width = 1
    while width < n_qubits:
        for i in range(0,width,1):
            if i+width > n_qubits-1:
                break
            circuit.append(cirq.CNOT(qubits[i], qubits[i+width]))
        width *= 2
    return circuit
print(build_log_ghz(7))


              ┌──┐   ┌───┐
0: ───H───@────@──────@──────
          │    │      │
1: ───────X────┼@─────┼@─────
               ││     ││
2: ────────────X┼─────┼┼@────
                │     │││
3: ─────────────X─────┼┼┼────
                      │││
4: ───────────────────X┼┼────
                       ││
5: ────────────────────X┼────
                        │
6: ─────────────────────X────
              └──┘   └───┘


In [26]:
circuit = build_log_ghz(7)
parallelized = utils.parallelize(circuit=circuit)
parallelized = utils.no_similarity(parallelized)
print(parallelized)

                                                            ┌──┐   ┌───────────────────────┐   ┌──────────────────────┐
0: ───PhXZ(a=0.5,x=0.5,z=0)────@───PhXZ(a=0.5,x=1,z=0)───────@───────────────────────────@────────────────────────────────────────────────────────
                               │                             │                           │
1: ───PhXZ(a=0.5,x=0.5,z=0)────@───PhXZ(a=0.5,x=0.5,z=-1)────┼@──────────────────────────┼@───────────────────────────────────────────────────────
                                                             ││                          ││
2: ───PhXZ(a=0.5,x=-0.5,z=0)─────────────────────────────────@┼─────PhXZ(a=0.5,x=0.5,z=0)┼┼─────@─────────────────────────────────────────────────
                                                              │                          ││     │
3: ───PhXZ(a=0.5,x=-0.5,z=0)──────────────────────────────────@─────PhXZ(a=0.5,x=0.5,z=0)┼┼─────┼─────────────────────────────────────────────────
           

/Users/jwurtz/Documents/github/QuEra/bloqade_unirepo/bloqade/.venv/lib/python3.10/site-packages/cirq/circuits/circuit_operation.py:173: FutureWarning: In cirq 1.6 the default value of `use_repetition_ids` will change to
`use_repetition_ids=False`. To make this warning go away, please pass
explicit `use_repetition_ids`, e.g., to preserve current behavior, use

  CircuitOperations(..., use_repetition_ids=True)
  warnings.warn(msg, FutureWarning)
/Users/jwurtz/Documents/github/QuEra/bloqade_unirepo/bloqade/.venv/lib/python3.10/site-packages/cirq/circuits/circuit_operation.py:173: FutureWarning: In cirq 1.6 the default value of `use_repetition_ids` will change to
`use_repetition_ids=False`. To make this warning go away, please pass
explicit `use_repetition_ids`, e.g., to preserve current behavior, use

  CircuitOperations(..., use_repetition_ids=True)
  warnings.warn(msg, FutureWarning)
/Users/jwurtz/Documents/github/QuEra/bloqade_unirepo/bloqade/.venv/lib/python3.10/site-packages/cirq/cir

### Directed acyclic graphs, integer linear programs, and similarity
- DAGs can be written as a series of inequalities, with each vertex $i$ given an integer label $t_i$ of its epoch.
- The constraints of the ILP are totally unimodular, so the solutions are always integers and thus efficient.
- Similarity can be represented as trying to minimize $|t_i-t_j|$ between two similar vertices $i$ and $j$ with weight $w_{ij}$
- Minimizing the total depth of the circuit is the same as minimizing the total time via a linear objective $\sum_i \lambda_i t_i$

Annotate similarity by tags, to be ingested by the parallelism functions

In [29]:
utils.auto_similarity   # Similarity if two gates are commuting and identical
utils.block_similarity  # Give all gates in the circuit the same similarity tag (useful for concatenating circuits together)
utils.moment_similarity # Give all gates in the same moment the same simiarlity tag
utils.no_similarity     # Remove all tags
utils.parallelize(circuit=circuit, auto_tag = False) # auto_tag = False if you want to manually tag similarity

/Users/jwurtz/Documents/github/QuEra/bloqade_unirepo/bloqade/.venv/lib/python3.10/site-packages/cirq/circuits/circuit_operation.py:173: FutureWarning: In cirq 1.6 the default value of `use_repetition_ids` will change to
`use_repetition_ids=False`. To make this warning go away, please pass
explicit `use_repetition_ids`, e.g., to preserve current behavior, use

  CircuitOperations(..., use_repetition_ids=True)
  warnings.warn(msg, FutureWarning)
/Users/jwurtz/Documents/github/QuEra/bloqade_unirepo/bloqade/.venv/lib/python3.10/site-packages/cirq/circuits/circuit_operation.py:173: FutureWarning: In cirq 1.6 the default value of `use_repetition_ids` will change to
`use_repetition_ids=False`. To make this warning go away, please pass
explicit `use_repetition_ids`, e.g., to preserve current behavior, use

  CircuitOperations(..., use_repetition_ids=True)
  warnings.warn(msg, FutureWarning)
/Users/jwurtz/Documents/github/QuEra/bloqade_unirepo/bloqade/.venv/lib/python3.10/site-packages/cirq/cir

┌──┐   ┌──┐
0: ───PhXZ(a=0.5,x=0.5,z=0)────@───PhXZ(a=0.5,x=1,z=0)───────@──────@─────────────────────────────────────────────────────────
                               │                             │      │
1: ───PhXZ(a=0.5,x=0.5,z=0)────@───PhXZ(a=0.5,x=0.5,z=-1)────┼@─────┼@────────────────────────────────────────────────────────
                                                             ││     ││
2: ───PhXZ(a=0.5,x=-0.5,z=0)─────────────────────────────────@┼─────┼┼────PhXZ(a=0.5,x=0.5,z=0)───@───────────────────────────
                                                              │     ││                            │
3: ───PhXZ(a=0.5,x=-0.5,z=0)──────────────────────────────────@─────┼┼────PhXZ(a=0.5,x=0.5,z=0)───┼───────────────────────────
                                                                    ││                            │
4: ───PhXZ(a=0.5,x=-0.5,z=0)────────────────────────────────────────@┼────PhXZ(a=0.5,x=0.5,z=0)───┼───────────────────────────
                                                                     │                            │
5: ───PhXZ(a=0.5,x=-0.5,z=0)─────────────────────────────────────────@────PhXZ(a=0.5,x=0.5,z=0)───┼───────────────────────────
                                                                                                  │
6: ───PhXZ(a=0.5,x=-0.5,z=0)──────────────────────────────────────────────────────────────────────@───PhXZ(a=0.5,x=0.5,z=0)───
                                                            └──┘   └──┘

### Performance of parallel-tuned circuits
Lets try out some circuits that have been passed through these parallelism functions on our noise models to see if adding parallelism helps.
- QAOA
- GHZ
- VQE
- Hypercube
- Random
  
Extend https://bloqade.quera.com/latest/digital/examples/interop/noisy_ghz/#studying-the-fidelity

In [31]:
noisy = utils.noise.transform_circuit(circuit,model = utils.noise.GeminiOneZoneNoiseModel())

/Users/jwurtz/Documents/github/QuEra/bloqade_unirepo/bloqade/.venv/lib/python3.10/site-packages/cirq/circuits/circuit_operation.py:173: FutureWarning: In cirq 1.6 the default value of `use_repetition_ids` will change to
`use_repetition_ids=False`. To make this warning go away, please pass
explicit `use_repetition_ids`, e.g., to preserve current behavior, use

  CircuitOperations(..., use_repetition_ids=True)
  warnings.warn(msg, FutureWarning)
/Users/jwurtz/Documents/github/QuEra/bloqade_unirepo/bloqade/.venv/lib/python3.10/site-packages/cirq/circuits/circuit_operation.py:173: FutureWarning: In cirq 1.6 the default value of `use_repetition_ids` will change to
`use_repetition_ids=False`. To make this warning go away, please pass
explicit `use_repetition_ids`, e.g., to preserve current behavior, use

  CircuitOperations(..., use_repetition_ids=True)
  warnings.warn(msg, FutureWarning)
/Users/jwurtz/Documents/github/QuEra/bloqade_unirepo/bloqade/.venv/lib/python3.10/site-packages/cirq/cir